In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from keras.models import Sequential
# from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
# from tensorflow.keras import initializers
# from tensorflow.keras import regularizers
# from tensorflow.keras.models import Sequential, save_model, load_model

In [2]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [3]:
demo = pd.read_csv("PTDEMOG.csv")

In [4]:
neuro = pd.read_csv("NEUROEXM.csv")

In [5]:
neuro.columns

Index(['Phase', 'ID', 'RID', 'SITEID', 'VISCODE', 'VISCODE2', 'USERDATE',
       'USERDATE2', 'EXAMDATE', 'NXVISUAL', 'NXVISDES', 'NXAUDITO', 'NXAUDDES',
       'NXTREMOR', 'NXTREDES', 'NXCONSCI', 'NXCONDES', 'NXNERVE', 'NXNERDES',
       'NXMOTOR', 'NXMOTDES', 'NXFINGER', 'NXFINDES', 'NXHEEL', 'NXHEEDES',
       'NXSENSOR', 'NXSENDES', 'NXTENDON', 'NXTENDES', 'NXPLANTA', 'NXPLADES',
       'NXGAIT', 'NXGAIDES', 'NXOTHER', 'NXOTHSPE', 'NXGENCOM', 'NXABNORM',
       'update_stamp'],
      dtype='object')

In [6]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [7]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [8]:
diag["Subject"].value_counts()

022_S_0096    4
027_S_0120    4
141_S_1378    4
131_S_0384    4
128_S_0272    4
             ..
012_S_1321    1
068_S_0872    1
016_S_1263    1
130_S_1290    1
016_S_6836    1
Name: Subject, Length: 2759, dtype: int64

In [9]:
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]

In [10]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [11]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [12]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [13]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [14]:
m["PTID"].value_counts()

067_S_4767    18
141_S_0697    15
131_S_0384    15
011_S_0023    14
126_S_1077    14
              ..
168_S_6064     1
011_S_4326     1
014_S_6087     1
129_S_6089     1
131_S_6153     1
Name: PTID, Length: 2411, dtype: int64

In [15]:
t = m

In [16]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE", "PTDOBMM","DX"], axis=1) 

In [17]:
t.columns

Index(['RID', 'PTID', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND', 'PTMARRY',
       'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTWRECNT', 'PTNOTRT', 'PTRTYR',
       'PTHOME', 'PTOTHOME', 'PTTLANG', 'PTPLANG', 'PTPSPEC', 'PTCOGBEG',
       'PTMCIBEG', 'PTADBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'EXAMDATE',
       'NXVISUAL', 'NXVISDES', 'NXAUDITO', 'NXAUDDES', 'NXTREMOR', 'NXTREDES',
       'NXCONSCI', 'NXCONDES', 'NXNERVE', 'NXNERDES', 'NXMOTOR', 'NXMOTDES',
       'NXFINGER', 'NXFINDES', 'NXHEEL', 'NXHEEDES', 'NXSENSOR', 'NXSENDES',
       'NXTENDON', 'NXTENDES', 'NXPLANTA', 'NXPLADES', 'NXGAIT', 'NXGAIDES',
       'NXOTHER', 'NXOTHSPE', 'NXGENCOM', 'NXABNORM', 'SUBJECT_KEY', 'AGE',
       'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [18]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [19]:
len(t.columns)

41

In [20]:
t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
                                                                                                                   "bookkeeper").str.replace("cpa", "accounting")

In [21]:
t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [22]:
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

C:\Users\Nithin KN\AppData\Local\Temp\ipykernel_10632\2309654543.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
C:\Users\Nithin KN\AppData\Local\Temp\ipykernel_10632\2309654543.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
C:\Users\Nithin KN\AppData\Local\Temp\ipykernel_10632\2309654543.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
C:\Users\Nithin KN\AppData\Local\Temp\ipykernel_10632\2309654543.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
C:\Users\Nithin KN\AppData\Local\Temp\ipykernel_1063

In [23]:
cond = t['PTWORK'].value_counts()
threshold = 10
t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [24]:
categorical = ['PTGENDER', 'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [25]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [26]:
text = ["PTWORK", "CMMED"]

In [27]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,PTRACCAT,NXABNORM,PHC_LAN,NXPLANTA,AGE,PTCOGBEG,PTRTYR,NXMOTOR,NXCONSCI,EXAMDATE,...,NXFINGER,NXOTHER,PHC_VSP,PHC_EXF,NXTENDON,PTTLANG,NXSENSOR,NXHEEL,PTADDX,PTDOBYY
0,5.0,1.0,-0.828,1.0,82.3655,-4.0,9/18/1989,1.0,1.0,2005-08-18,...,1.0,1.0,0.128,-1.036,1.0,1.0,1.0,1.0,-4.0,1924.0
1,5.0,1.0,-0.516,1.0,81.3662,-4.0,9/18/1989,1.0,1.0,2005-08-18,...,1.0,1.0,-0.237,-0.670,1.0,1.0,1.0,1.0,-4.0,1924.0
2,5.0,1.0,-0.756,1.0,83.3648,-4.0,9/18/1989,1.0,1.0,2005-08-18,...,1.0,1.0,-4.000,-0.645,1.0,1.0,1.0,1.0,-4.0,1924.0
3,5.0,1.0,-0.516,1.0,81.8645,-4.0,9/18/1989,1.0,1.0,2005-08-18,...,1.0,1.0,-0.305,-0.145,1.0,1.0,1.0,1.0,-4.0,1924.0
4,5.0,1.0,0.362,1.0,69.3634,-4.0,-4,1.0,1.0,2005-08-18,...,1.0,1.0,-0.868,-0.662,1.0,2.0,1.0,1.0,-4.0,1938.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7741,5.0,1.0,-4.000,1.0,80.9801,2017.0,2009-xx-xx,1.0,1.0,-4,...,1.0,1.0,-4.000,-4.000,1.0,1.0,1.0,1.0,9999.0,1938.0
7742,5.0,1.0,-4.000,1.0,76.5996,2009.0,2009-xx-xx,1.0,1.0,-4,...,1.0,-4.0,-4.000,-4.000,2.0,1.0,1.0,1.0,9999.0,1943.0
7743,5.0,2.0,-4.000,-4.0,74.7598,2014.0,2019-02-xx,-4.0,-4.0,-4,...,-4.0,-4.0,-4.000,-4.000,-4.0,1.0,-4.0,-4.0,-4.0,1945.0
7744,5.0,1.0,-4.000,1.0,74.6804,2019.0,2007-11-30,1.0,1.0,-4,...,2.0,2.0,-4.000,-4.000,1.0,1.0,1.0,1.0,2019.0,1945.0


In [28]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

PTRACCAT
NXABNORM
NXPLANTA
NXMOTOR
NXCONSCI
NXNERVE
NXGAIT
PTETHCAT
NXFINGER
NXOTHER
NXTENDON
PTTLANG
NXSENSOR
NXHEEL


In [29]:
# to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]
to_del = ["PTRTYR","SUBJECT_KEY", "PTWRECNT"]
t = t.drop(to_del, axis=1)

In [30]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

,PHC_VSP,PHC_LAN,PTCOGBEG,AGE,PHC_EXF,EXAMDATE,PHC_MEM,PTADDX,PTDOBYY
0,0.128,-0.828,-4.0,82.3655,-1.036,2005-08-18,-0.973,-4.0,1924.0
1,-0.237,-0.516,-4.0,81.3662,-0.670,2005-08-18,-1.068,-4.0,1924.0
2,-4.000,-0.756,-4.0,83.3648,-0.645,2005-08-18,-1.559,-4.0,1924.0
3,-0.305,-0.516,-4.0,81.8645,-0.145,2005-08-18,-0.768,-4.0,1924.0
4,-0.868,0.362,-4.0,69.3634,-0.662,2005-08-18,0.280,-4.0,1938.0
...,...,...,...,...,...,...,...,...,...
7741,-4.000,-4.000,2017.0,80.9801,-4.000,-4,0.624,9999.0,1938.0
7742,-4.000,-4.000,2009.0,76.5996,-4.000,-4,-0.180,9999.0,1943.0
7743,-4.000,-4.000,2014.0,74.7598,-4.000,-4,0.870,-4.0,1945.0
7744,-4.000,-4.000,2019.0,74.6804,-4.000,-4,-0.814,2019.0,1945.0


In [31]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [32]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [33]:
t[l]

,RID,PTID,Group,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTWORK,...,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.973,-1.036,-0.828,0.128
1,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.068,-0.670,-0.516,-0.237
2,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.559,-0.645,-0.756,-4.000
3,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.768,-0.145,-0.516,-0.305
4,4,022_S_0004,1.0,ADNI1,1.0,1938.0,1.0,1.0,10.0,other,...,1.0,1.0,1.0,1.0,1.0,1.0,0.280,-0.662,0.362,-0.868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7741,6725,013_S_6725,1.0,ADNI3,1.0,1938.0,1.0,3.0,20.0,other,...,1.0,1.0,1.0,1.0,1.0,1.0,0.624,-4.000,-4.000,-4.000
7742,6851,168_S_6851,1.0,ADNI3,2.0,1943.0,1.0,3.0,18.0,nurse,...,1.0,1.0,2.0,1.0,2.0,1.0,-0.180,-4.000,-4.000,-4.000
7743,6841,035_S_6841,0.0,ADNI3,2.0,1945.0,1.0,1.0,14.0,sales,...,-4.0,-4.0,-4.0,-4.0,-4.0,2.0,0.870,-4.000,-4.000,-4.000
7744,6855,032_S_6855,2.0,ADNI3,1.0,1945.0,1.0,1.0,16.0,analyst,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.814,-4.000,-4.000,-4.000


In [34]:
dfs = []

In [35]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [36]:
cat = pd.concat(dfs, axis=1)

In [37]:
t[quant]

,PHC_VSP,PHC_LAN,PTCOGBEG,AGE,PHC_EXF,EXAMDATE,PHC_MEM,PTADDX,PTDOBYY
0,0.128,-0.828,-4.0,82.3655,-1.036,2005-08-18,-0.973,-4.0,1924.0
1,-0.237,-0.516,-4.0,81.3662,-0.670,2005-08-18,-1.068,-4.0,1924.0
2,-4.000,-0.756,-4.0,83.3648,-0.645,2005-08-18,-1.559,-4.0,1924.0
3,-0.305,-0.516,-4.0,81.8645,-0.145,2005-08-18,-0.768,-4.0,1924.0
4,-0.868,0.362,-4.0,69.3634,-0.662,2005-08-18,0.280,-4.0,1938.0
...,...,...,...,...,...,...,...,...,...
7741,-4.000,-4.000,2017.0,80.9801,-4.000,-4,0.624,9999.0,1938.0
7742,-4.000,-4.000,2009.0,76.5996,-4.000,-4,-0.180,9999.0,1943.0
7743,-4.000,-4.000,2014.0,74.7598,-4.000,-4,0.870,-4.0,1945.0
7744,-4.000,-4.000,2019.0,74.6804,-4.000,-4,-0.814,2019.0,1945.0


In [38]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTWORK_accounting,PTWORK_admin,PTWORK_analyst,PTWORK_architect,PTWORK_army officer,PTWORK_artist,...,NXTENDON_2.0,PTTLANG_-4.0,PTTLANG_1.0,PTTLANG_2.0,NXSENSOR_-4.0,NXSENSOR_1.0,NXSENSOR_2.0,NXHEEL_-4.0,NXHEEL_1.0,NXHEEL_2.0
0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7741,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
7742,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
7743,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
7744,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [39]:
t[["PTID","RID", "Phase", "Group"]]

,PTID,RID,Phase,Group
0,011_S_0003,3,ADNI1,2.0
1,011_S_0003,3,ADNI1,2.0
2,011_S_0003,3,ADNI1,2.0
3,011_S_0003,3,ADNI1,2.0
4,022_S_0004,4,ADNI1,1.0
...,...,...,...,...
7741,013_S_6725,6725,ADNI3,1.0
7742,168_S_6851,6851,ADNI3,1.0
7743,035_S_6841,6841,ADNI3,0.0
7744,032_S_6855,6855,ADNI3,2.0


In [40]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [41]:
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTWORK_accounting,PTWORK_admin,...,NXHEEL_2.0,PHC_VSP,PHC_LAN,PTCOGBEG,AGE,PHC_EXF,EXAMDATE,PHC_MEM,PTADDX,PTDOBYY
0,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,0.128,-0.828,-4.0,82.3655,-1.036,2005-08-18,-0.973,-4.0,1924.0
1,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,-0.237,-0.516,-4.0,81.3662,-0.670,2005-08-18,-1.068,-4.0,1924.0
2,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,-4.000,-0.756,-4.0,83.3648,-0.645,2005-08-18,-1.559,-4.0,1924.0
3,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,-0.305,-0.516,-4.0,81.8645,-0.145,2005-08-18,-0.768,-4.0,1924.0
4,022_S_0004,4,ADNI1,1.0,0,1,0,0,0,0,...,0,-0.868,0.362,-4.0,69.3634,-0.662,2005-08-18,0.280,-4.0,1938.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7741,013_S_6725,6725,ADNI3,1.0,0,1,0,0,0,0,...,0,-4.000,-4.000,2017.0,80.9801,-4.000,-4,0.624,9999.0,1938.0
7742,168_S_6851,6851,ADNI3,1.0,0,0,1,0,0,0,...,0,-4.000,-4.000,2009.0,76.5996,-4.000,-4,-0.180,9999.0,1943.0
7743,035_S_6841,6841,ADNI3,0.0,0,0,1,0,0,0,...,0,-4.000,-4.000,2014.0,74.7598,-4.000,-4,0.870,-4.0,1945.0
7744,032_S_6855,6855,ADNI3,2.0,0,1,0,0,0,0,...,0,-4.000,-4.000,2019.0,74.6804,-4.000,-4,-0.814,2019.0,1945.0


In [42]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [43]:
c.to_csv("clinical.csv")
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTWORK_accounting,PTWORK_admin,...,NXHEEL_2.0,PHC_VSP,PHC_LAN,PTCOGBEG,AGE,PHC_EXF,EXAMDATE,PHC_MEM,PTADDX,PTDOBYY
0,002_S_0295,295,ADNI1,0.0,0,1,0,0,0,0,...,0,0.483,0.272,-4.0,84.9363,0.554,2006-04-04,1.357,-4.0,1921.0
1,002_S_0413,413,ADNI1,0.0,0,0,1,0,1,0,...,0,-4.000,2.594,-4.0,76.9528,1.480,2006-04-06,1.287,-4.0,1929.0
2,002_S_0559,559,ADNI1,0.0,0,1,0,0,0,0,...,0,-4.000,1.041,-4.0,80.5284,1.176,2006-05-11,0.768,-4.0,1927.0
3,002_S_0619,619,ADNI1,2.0,0,1,0,0,0,0,...,0,0.264,0.116,-4.0,78.0315,-0.509,2006-05-18,-1.259,-4.0,1928.0
4,002_S_0685,685,ADNI1,0.0,0,0,1,0,0,0,...,0,-4.000,1.170,-4.0,92.7228,0.111,2006-06-22,0.149,-4.0,1916.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2406,941_S_6580,6580,ADNI3,0.0,0,0,1,0,0,0,...,0,-0.583,0.159,9999.0,79.9535,-0.208,-4,0.082,9999.0,1938.0
2407,941_S_6581,6581,ADNI3,0.0,0,0,1,0,0,0,...,0,-4.000,1.281,9999.0,74.5489,0.824,-4,1.142,9999.0,1944.0
2408,941_S_6607,6607,ADNI3,0.0,0,1,0,0,0,1,...,0,-4.000,-4.000,9999.0,82.7871,-4.000,-4,0.624,9999.0,1936.0
2409,941_S_6803,6803,ADNI3,1.0,0,0,1,0,0,0,...,0,-0.041,-0.147,2014.0,75.1266,-0.854,-4,-0.604,9999.0,1944.0


In [44]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTWORK_accounting,PTWORK_admin,...,NXHEEL_2.0,PHC_VSP,PHC_LAN,PTCOGBEG,AGE,PHC_EXF,EXAMDATE,PHC_MEM,PTADDX,PTDOBYY
0,002_S_0295,295,ADNI1,0.0,0,1,0,0,0,0,...,0,0.483,0.272,-4.0,84.9363,0.554,2006-04-04,1.357,-4.0,1921.0
2,002_S_0559,559,ADNI1,0.0,0,1,0,0,0,0,...,0,-4.000,1.041,-4.0,80.5284,1.176,2006-05-11,0.768,-4.0,1927.0
3,002_S_0619,619,ADNI1,2.0,0,1,0,0,0,0,...,0,0.264,0.116,-4.0,78.0315,-0.509,2006-05-18,-1.259,-4.0,1928.0
4,002_S_0685,685,ADNI1,0.0,0,0,1,0,0,0,...,0,-4.000,1.170,-4.0,92.7228,0.111,2006-06-22,0.149,-4.0,1916.0
5,002_S_0729,729,ADNI1,2.0,0,0,1,0,0,0,...,0,0.264,0.364,-4.0,65.8125,0.608,2006-07-06,-0.016,-4.0,1941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2406,941_S_6580,6580,ADNI3,0.0,0,0,1,0,0,0,...,0,-0.583,0.159,9999.0,79.9535,-0.208,-4,0.082,9999.0,1938.0
2407,941_S_6581,6581,ADNI3,0.0,0,0,1,0,0,0,...,0,-4.000,1.281,9999.0,74.5489,0.824,-4,1.142,9999.0,1944.0
2408,941_S_6607,6607,ADNI3,0.0,0,1,0,0,0,1,...,0,-4.000,-4.000,9999.0,82.7871,-4.000,-4,0.624,9999.0,1936.0
2409,941_S_6803,6803,ADNI3,1.0,0,0,1,0,0,0,...,0,-0.041,-0.147,2014.0,75.1266,-0.854,-4,-0.604,9999.0,1944.0


In [45]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [46]:
# X_train.to_pickle("X_train_c.pkl")
# y_train.to_pickle("y_train_c.pkl")

# X_test.to_pickle("X_test_c.pkl")
# y_test.to_pickle("y_test_c.pkl")

with open("X_train_c.pkl",'wb') as f:
    np.save(f,X_train)
with open("y_train_c.pkl",'wb') as f:
    np.save(f,y_train)
with open("X_test_c.pkl",'wb') as f:
    np.save(f,X_test)
with open("y_test_c.pkl",'wb') as f:
    np.save(f,y_test)